In [ ]:
# Install required packages
!pip install -qU crewai[tools]
!pip install -qU tavily-python scrapegraph-py
!pip install --upgrade litellm==1.60.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.7 MB/s eta 0:0

In [ ]:
import os
import json
import sqlite3
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import time
import re
from contextlib import contextmanager
from google.colab import files
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from pydantic import BaseModel, Field
from typing import List
from typing import List, Dict, Any, Optional
from scrapegraph_py import Client
from tavily import TavilyClient
from google.colab import userdata
from crewai_tools import SerperDevTool

In [ ]:
# Define API keys dictionary for centralized management
api_keys = {
    "OPENAI_API_KEY": "Open_API_Key",
    "TAVILY_API_KEY": "Tavely_API_Key",
    "SERPER_API_KEY": "Serper_API_Key",
    "SCRAPEGRAPH_API_KEY": "scrapegraph"
}

# Set environment variables from userdata
for env_var, key_name in api_keys.items():
    os.environ[env_var] = userdata.get(key_name)

# Set other environment variables
os.environ["LITELLM_LOG"] = "DEBUG"

# Log success without exposing keys
print(f"API keys loaded: {', '.join([k for k, v in api_keys.items() if os.environ.get(k)])}")

API keys loaded: OPENAI_API_KEY, TAVILY_API_KEY, SERPER_API_KEY, SCRAPEGRAPH_API_KEY


In [ ]:
# Initialize AI tools
search_client = TavilyClient(api_key=userdata.get('Tavely_API_Key'))
google_search_tool = SerperDevTool()
scrape_client = Client(api_key=userdata.get('scrapegraph'))

In [ ]:
# Create output directory
output_dir = "./trip-planning-output"
os.makedirs(output_dir, exist_ok=True)

# Configure LLM
import litellm
litellm.set_verbose = True  # Enable debugging
basic_llm = LLM(
    model="gpt-4o",
    temperature=0)

In [ ]:
# Create timing decorator for performance monitoring
@contextmanager
def timing(operation_name):
    """Context manager for timing operations"""
    start_time = time.time()
    yield
    elapsed_time = time.time() - start_time
    print(f"⏱️ {operation_name} completed in {elapsed_time:.2f} seconds")

"""# 🔍 1. SEARCH QUERY GENERATOR AGENT"""

In [ ]:
# Number of keywords to generate
no_keywords = 10

# Create SuggestedTripQueries class dynamically
def create_suggested_trip_queries(max_keywords):
    class SuggestedTripQueries(BaseModel):
        queries: List[str] = Field(...,
                                   title="Optimized search queries for best hotels",
                                   min_items=1,
                                   max_items=max_keywords)
    return SuggestedTripQueries

# Create the class with current value of no_keywords
SuggestedTripQueries = create_suggested_trip_queries(no_keywords)

search_query_agent = Agent(
    role="Search Query Generator Agent",
    goal="\n".join([
        "Generate optimized search queries for booking websites to find the best hotels deals.",
        "Queries should focus on deals within the given budget ({budget}) and duration ({trip_duration} days) and trip type ({trip_type}).",
        "Ensure that search queries reflect the traveler's preferences for cost and trip length and trip type.",
        "Provide a mix of options (e.g., budget, luxury, and mid-range)."
    ]),
    backstory="This agent specializes in looking for best hotels deals by providing a list of suggested search queries to be passed to the search engine based on location, budget, and trip type.",
    llm=basic_llm,
    verbose=True,
)

search_query_task = Task(
    description="\n".join([
        "Generate up to {no_keywords} search queries for hotel packages.",
        "Ensure queries are optimized for specific booking websites: {websites_list}.",
        "Include the following constraints:",
        "1️⃣ Country: {country_name}",
        "2️⃣ City: {city_name}",
        "3️⃣ Budget range: {budget}",
        "4️⃣ Trip duration: {trip_duration} days",
        "Trip type: {trip_type}",
        "The queries should focus on deals within the specified budget and duration.",
        "Avoid general keywords."
    ]),
    expected_output="A JSON object containing a list of optimized search queries.",
    output_json=SuggestedTripQueries,
    output_file=os.path.join(output_dir, "step_1_search_queries.json"),
    agent=search_query_agent
)

/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1011: PydanticDeprecatedSince20: `min_items` is deprecated and will be removed, use `min_length` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warn('`min_items` is deprecated and will be removed, use `min_length` instead', DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1017: PydanticDeprecatedSince20: `max_items` is deprecated and will be removed, use `max_length` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warn('`max_items` is deprecated and will be removed, use `max_length` instead', DeprecationWarning)


"""# 🌐 2. SEARCH ENGINE AGENT"""

In [ ]:
class SearchResult(BaseModel):
    title: str
    url: str = Field(..., title="The page URL")
    snippet: str
    score: float
    search_query: str

class AllSearchResults(BaseModel):
    results: List[SearchResult]

@lru_cache(maxsize=32)
def cached_search(query: str):
    """Cache search results to avoid duplicate API calls"""
    return search_client.search(query)

@tool
def search_engine_tool(query: str):
    """Useful for search-based queries. Use this to find current information about any query related pages using a search engine"""
    try:
        with timing(f"Search for '{query}'"):
            response = cached_search(query)

        if not response:
            return {"error": "No results found", "query": query}

        return response
    except ConnectionError as e:
        return {"error": f"Connection error: {str(e)}", "query": query}
    except TimeoutError as e:
        return {"error": f"Request timed out: {str(e)}", "query": query}
    except Exception as e:
        return {"error": f"Search API failed: {str(e)}", "query": query, "error_type": type(e).__name__}

search_engine_agent = Agent(
    role="Search Engine Agent",
    goal="\n".join([
        "Search for hotel packages and hotels in {country_name} and {city_name} using provided search queries.",
        "Ensure results match the budget ({budget}) and trip type ({trip_type}).",
        "NEVER use the same tool twice in a row.",
        "Your response must follow this format exactly:",
        "Thought: Your reasoning",
        "Action: search_engine_tool or google_search_tool (only if needed)",
        "Final Answer: A clear, structured response with hotel deals."
    ]),
    backstory="A professional hotel research agent that retrieves the best booking offers.",
    llm=basic_llm,
    tools=[search_engine_tool, google_search_tool],
    verbose=True)

search_engine_task = Task(
    description="Use the search engine tool to find hotel deals in {country_name} and {city_name} based on the generated search queries. "
                "Ignore blogs, ads, or unrelated links.",
    expected_output="A JSON object containing search results with hotel deals.",
    output_json=AllSearchResults,
    output_file=os.path.join(output_dir, "step_2_search_results.json"),
    agent=search_engine_agent
)

# 🕵️ 3. WEB SCRAPING AGEN

In [ ]:
class HotelSpec(BaseModel):
    specification_name: str
    specification_value: str

class TripPackageDetails(BaseModel):
    page_url: str = Field(..., title="The page URL of the hotel page")
    hotel_name: str = Field(..., title="The name of the hotel")
    hotel_description: str = Field(..., title="A brief description of the hotel")
    hotel_address: str = Field(..., title="The address of the hotel")
    location: str = Field(..., title="The location of the hotel")
    rating: float = Field(title="The rating of the hotel based on reviews", default=None)
    reviews: int = Field(title="The number of reviews for the hotel", default=None)
    price: float = Field(title="The price of the hotel package", default=None)
    currency: str = Field(title="The currency of the price", default=None)
    included_meals: str = Field(title="The meals included in the package", default= None)
    excluded_meals: str = Field(title="The meals excluded from the package", default= None)
    facilities: str = Field(title="The facilities which the hotel provides to their guests", default= None)
    #room_type = str = Field (title= "The type of room options like single room, double room, or family room ", default=None)

    product_specs: List[HotelSpec] = Field(..., title="The specifications of the hotel. Focus on the most important specs to compare.", min_items=1, max_items=5)
    agent_recommendation_rank: int = Field(..., title="The rank of the hotel to be considered in the final procurement report. (out of 5, Higher is Better) in the recommendation list ordering from the best to the worst")
    agent_recommendation_notes: List[str]  = Field(..., title="A set of notes why would you recommend or not recommend this hotel to the traveller, compared to other hotels.")


class AllTripPackages(BaseModel):
    packages: List[TripPackageDetails]

@tool
def web_scraping_tool(page_url: str, timeout=30):
    """An AI tool to scrape hotel details and ensure the location matches the search location.
       Example: web_scraping_tool (page_url = "https://www.booking.com/family/city/eg/luxor.html")
    """
    try:
        with timing(f"Scraping {page_url}"):
            details = scrape_client.smartscraper(
                website_url=page_url,
                user_prompt=f"Extract ```json\n" + TripPackageDetails.schema_json() + "```\n From the web page"
                #timeout=timeout
            )

        return {"page_url": page_url, "details": details}

    except TimeoutError:
        return {"error": f"Scraping timed out after {timeout} seconds", "page_url": page_url}
    except Exception as e:
        return {"error": f"Scraping failed: {str(e)}", "page_url": page_url}



scraping_agent = Agent(
    role="Web Scraping Agent",
    goal="Extract hotel details and related activities from search results pages generated by search_engine_agent.",
    backstory="The agent is designed to help in looking for required values from any website url. These details will be used to decide which best hotel to book.",
    llm=basic_llm,
    tools=[web_scraping_tool],
    verbose=True,
)

scraping_task = Task(
      description="\n".join([
                "Extract hotel package details including hotel name, price, rating, reviews, meals, and activities from any booking hotel page url.",
                "The task has to collect results from multiple pages urls.",
                "Collect the best top recommendations hotels from the search results."]),
      expected_output="A JSON object containing structured hotel package details.",
      output_json=AllTripPackages,
      output_file=os.path.join(output_dir, "step_3_trip_packages.json"),
       agent=scraping_agent,
      async_execution=True  # Run in parallel for better performance
)

In [ ]:
hotel_report_agent = Agent(
    role="Hotel Report Author Agent",
    goal="To generate a professional HTML page for the Hotel recommendation report",
    backstory="The agent is designed to assist in generating a professional HTML page for the  Hotel recommendation report after looking into a list of hotels.",
    llm=basic_llm,
    verbose=True,
)

hotel_report_task = Task(
    description="\n".join([
        "The task is to generate a professional HTML page for the Hotel recommendation report.",
        "You have to use Bootstrap CSS framework for a better UI.",
        "The report will include the search results and prices of hotels from different websites.",
        "The report should be structured with the following sections:",
        "1. Executive Summary: A brief overview of the hotel process and key findings.",
        "2. Introduction: An introduction to the purpose and scope of the report.",
        "3. Methodology: A description of the methods used to gather and compare prices.",
        "4. Findings: Detailed comparison of prices from different websites, including tables and charts.",
        "5. Analysis: An analysis of the findings, highlighting any significant trends or observations.",
        "6. Recommendations: Suggestions for Hotel based on the analysis.",
        "7. Conclusion: A summary of the report and final thoughts.",
        "8. Appendices: Any additional information, such as raw data or supplementary materials.",
    ]),

    expected_output="A professional HTML page for the Hotel recommendation report.",
    output_file=os.path.join(output_dir, "step_4_trip_report.html"),
    agent=hotel_report_agent,
)

# 🚀 ASSEMBLING THE CREW

In [ ]:
trip_planning_crew = Crew(
    agents=[
        search_query_agent,
        search_engine_agent,
        scraping_agent,
        hotel_report_agent,
    ],
    tasks=[
        search_query_task,
        search_engine_task,
        scraping_task,
        hotel_report_task,
    ],
    process=Process.sequential,
    verbose=True
)

In [ ]:
# Enhanced processing function to prepare data for Streamlit
def process_results_for_streamlit(trip_packages, country_name, city_name, budget, trip_type, trip_duration):
    """Process the results into a format that's easy for Streamlit to display"""

    # Create a summary JSON
    summary = {
        "trip_details": {
            "country": country_name,
            "city": city_name,
            "budget": budget,
            "trip_type": trip_type,
            "duration": f"{trip_duration} days"
        },
        "hotel_count": len(trip_packages.get("packages", [])),
        "price_range": {
            "min": min([p.get("price", 0) for p in trip_packages.get("packages", []) if p.get("price")], default=0),
            "max": max([p.get("price", 0) for p in trip_packages.get("packages", []) if p.get("price")], default=0)
        },
        "top_hotels": sorted(trip_packages.get("packages", []),
                            key=lambda x: x.get("rating", 0),
                            reverse=True)[:3] if trip_packages.get("packages") else []
    }

    # Save the summary to a file
    with open(os.path.join(output_dir, "trip_summary.json"), "w") as f:
        json.dump(summary, f, indent=2)

    return summary

In [ ]:
def validate_budget(budget):
    """Validate budget format (e.g., $200-$1000 or 200-1000)"""
    return bool(re.match(r'^\$?\d+\s*-\s*\$?\d+$', budget))

def validate_duration(duration):
    """Validate duration format (e.g., 3 days or 3)"""
    # Extract just the number for consistent processing
    match = re.match(r'^(\d+)(?:\s*days?)?$', duration)
    if match:
        return int(match.group(1)) > 0
    return False

def get_valid_input(prompt, valid_options=None, validator_func=None):
    """Enhanced input validation with custom validator functions"""
    while True:
        user_input = input(prompt).strip()

        if not user_input:
            print("Input cannot be empty. Please enter a value.")
            continue

        if valid_options and user_input not in valid_options:
            print(f"Invalid input. Please choose from the following options: {', '.join(valid_options)}")
            continue

        if validator_func and not validator_func(user_input):
            print("Invalid input format. Please try again.")
            continue

        return user_input

In [ ]:
def normalize_duration(duration):
    """Extract the number from duration input"""
    match = re.match(r'^(\d+)(?:\s*days?)?$', duration)
    if match:
        return match.group(1)
    return duration

def download_output_files():
    """Function to download output files from Google Colab"""
    from google.colab import files
    import os

    output_dir = "./trip-planning-output"
    file_paths = [
        os.path.join(output_dir, "step_1_search_queries.json"),
        os.path.join(output_dir, "step_2_search_results.json"),
        os.path.join(output_dir, "step_3_trip_packages.json"),
        os.path.join(output_dir, "step_4_trip_report.html"),
        os.path.join(output_dir, "trip_summary.json")  # Added this file
    ]

    print("Downloading output files...")
    for file_path in file_paths:
        if os.path.exists(file_path):
            files.download(file_path)
            print(f"Downloaded: {file_path}")
        else:
            print(f"File not found: {file_path}")

In [ ]:
def run_trip_adviser():
    """Main function to run the Trip Adviser application"""
    print("=== Trip Adviser AI ===")
    print("Let's plan your perfect hotel stay!")

    # Get user inputs with validation
    country_name = get_valid_input("Enter the destination country: ")
    city_name = get_valid_input("Enter the desired city: ")

    budget = get_valid_input(
        "Enter your budget (e.g., $200-$1000): ",
        validator_func=validate_budget
    )

    trip_type = get_valid_input(
        "Enter your trip type (e.g., adventure, luxury, family): "
    )

    trip_duration_input = get_valid_input(
        "Enter your trip duration (e.g., 3 days): ",
        validator_func=validate_duration
    )
    trip_duration = normalize_duration(trip_duration_input)

    print("\nStarting hotel search process. This may take several minutes...\n")
    # Execute crew with timing
    with timing("Trip planning process"):
        try:
            crew_results = trip_planning_crew.kickoff(
                inputs={
                    "websites_list": ["Booking.com", "TripAdvisor", "Agoda", "Wego", "Expedia"],
                    "no_keywords": no_keywords,
                    "country_name": country_name,
                    "city_name": city_name,
                    "budget": budget,
                    "trip_type": trip_type,
                    "trip_duration": trip_duration
                }
            )
            print("\n✅ Trip Planning AI Crew completed!")
        except Exception as e:
            print(f"\n❌ Error during trip planning: {str(e)}")
            return None

    # Load the trip packages for processing
    trip_packages_file = os.path.join(output_dir, "step_3_trip_packages.json")
    if os.path.exists(trip_packages_file):
        try:
            with open(trip_packages_file, "r") as f:
                trip_packages = json.load(f)

            # Process results for Streamlit
            summary = process_results_for_streamlit(
                trip_packages, country_name, city_name, budget, trip_type, trip_duration
            )
            print(f"\nFound {summary['hotel_count']} hotels within your budget range.")
            if summary['hotel_count'] > 0:
                print(f"Price range: ${summary['price_range']['min']} - ${summary['price_range']['max']}")
            else:
                print("No hotels found within your criteria. Consider adjusting your search parameters.")
        except Exception as e:
            print(f"\n❌ Error processing results: {str(e)}")
    else:
        print("\n⚠️ No trip packages file found. The search might not have returned any results.")

    print(f"\nYou can find all outputs in the '{output_dir}' directory")

    # Add download option
    download_files = get_valid_input("Would you like to download the output files? (yes/no): ",
                                     valid_options=["yes", "no"])
    if download_files.lower() == "yes":
        download_output_files()

    return crew_results

In [ ]:
if __name__ == "__main__":
    run_trip_adviser()

=== Trip Adviser AI ===
Let's plan your perfect hotel stay!
Enter the destination country: Egypt
Enter the desired city: Hurghada
Enter your budget (e.g., $200-$1000): $200-$800
Enter your trip type (e.g., adventure, luxury, family): family
Enter your trip duration (e.g., 3 days): 4

Starting hotel search process. This may take several minutes...



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ad8e5ae1-5343-415f-bc26-b95894016c11                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: bcf4ba67-2cd5-45cd-9a30-27a6a5611dcf
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: bcf4ba67-2cd5-45cd-9a30-27a6a5611dcf
       Status: Executing Task...
    └── 🤖 Agent: Search Query Generator Agent
            Status: In Progress

# Agent: Search Query Generator Agent
## Task: Generate up to 10 search queries for hotel packages.
Ensure queries are optimized for specific booking websites: ['Booking.com', 'TripAdvisor', 'Agoda', 'Wego', 'Expedia'].
Include the following constraints:
1️⃣ Country: Egypt
2️⃣ City: Hurghada
3️⃣ Budget range: $200-$800
4️⃣ Trip duration: 4 days
Trip type: family
The queries should focus on deals within the specified budget and duration.
Avoid general keywords.


...

<IPython.core.display.Javascript object>

Downloaded: ./trip-planning-output/step_4_trip_report.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded: ./trip-planning-output/trip_summary.json
